In [1]:
from rich import print

from aimq.worker import Worker
from aimq.helpers import assign, const, pick, orig, echo, select
from aimq.tools.supabase import WriteRecord, ReadRecord, ReadFile, WriteFile, Enqueue
from aimq.tools.pdf import PageSplitter

In [2]:
worker = Worker()

In [3]:
@worker.task()
def process_document(data: dict):
    return (
        ReadRecord(
            table="documents_with_metadata",
            select="id, path:storage_object_path"
        )|
        ReadFile(bucket="files") |
        PageSplitter() |
        (
            WriteFile(
                bucket="pages",
                path="{{config.id}}/{{position}}/full.{{file.extension}}",
            ) | 
            assign({ "original": orig() }) |
            select("data") | 
            Enqueue(queue="process_page")
        ).map()
    )


@worker.task()
def process_page(data: dict):
    return (echo)

worker.send("process_document", { "id": "44e316a5-50ba-4698-aca1-a567d00ef4a8" })


19

In [4]:
worker.start(block=False)

In [5]:
worker.log(block=False)
worker.stop()

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ℹ️ INFO: Registered task process_document                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ℹ️ INFO: Registered task process_page                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ℹ️ INFO: Sent job 19 to queue process_document                                                                   │
│ Data: {'id': '44e316a5-50ba-4698-aca1-a567d00ef4a8'}                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ ℹ️ INFO: Worker thread started                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯